In [2]:
import osmnx
import geopandas as gpd

In [3]:
# loading trees data
trees_df = gpd.read_file("../data/SF_urban_tree_canopy.geojson")

In [4]:
# filtering ways which are considered as streets (see: https://wiki.openstreetmap.org/wiki/Key:highway)
osm_tags = {
    'highway': [
    "motorway",
    "trunk",
    "primary",
    "secondary",
    "living_street",
    "primary_link",
    "service",
    "unclassified",
    "motorway_link",
    "secondary_link",
    "tertiary",
    "residential",
    "trunk_link",
    "tertiary_link",
    "cycleway",
    "path",
    "track"
]
}

# loading street data
streets_df = osmnx.geometries_from_place("San Francisco", tags=osm_tags)

In [5]:
trees_df.head()

,acres,name,t_sqft,geometry
0,0.00000221382430071,Trees,0.0964338008024,"MULTIPOLYGON (((-122.43470 37.70830, -122.4347..."
1,0.000145516346336,Trees,6.33866669167,"MULTIPOLYGON (((-122.43473 37.70830, -122.4347..."
2,0.00012630908548,Trees,5.50200175546,"MULTIPOLYGON (((-122.43564 37.70829, -122.4356..."
3,0.00115850942522,Trees,50.464468704,"MULTIPOLYGON (((-122.43574 37.70831, -122.4357..."
4,0.00565374316464,Trees,246.276067144,"MULTIPOLYGON (((-122.43538 37.70831, -122.4353..."


In [6]:
# getting geometry columns
# that's what is important in case of intersection
street_shapes = streets_df.geometry

# buffer(0) deals with invalid trees shapes
trees_shapes = trees_df.geometry.buffer(0)

C:\Users\JF\.conda\envs\DataEngineering\lib\site-packages\geopandas\_vectorized.py:786: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'Polygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [
C:\Users\JF\.conda\envs\DataEngineering\lib\site-packages\geopandas\_vectorized.py:786: DeprecationWarning: An exception was ignored while fetching the attribute `__array_interface__` from an object of type 'MultiPolygon'.  With the exception of `AttributeError` NumPy will always raise this exception in the future.  Raise this deprecation warning to see the original exception. (Warning added NumPy 1.21)
  out[:] = [
C:\Users\JF\.conda\envs\DataEngineering\lib\site-packages\geopandas\_vectorized.py:142: DeprecationWarning: An exception was ignored while fetching the attribute `__

In [7]:
# checking if the shape is same
print(trees_shapes.shape,
    trees_df.geometry.shape)

(289219,) (289219,)


In [11]:
# calculating length of each way
streets_df['length'] = streets_df.geometry.length
streets_df['length'].head()

C:\Users\JF\AppData\Local\Temp\ipykernel_556\2226115096.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  streets_df['length'] = streets_df.geometry.length


element_type  osmid  
way           5004035    0.000851
              5071582    0.002131
              7373728    0.000986
              7373736    0.005475
              7448875    0.001665
Name: length, dtype: float64

## Calculating the shading of each way
Using the `intersection()` method, we calculate the common parts of trees and streets, obtaining several non-empty LINESTRING elements for each street that represent the shaded parts of the street. We then take the sum of the lengths of these elements and store this value in the `shaded_length` column. We can do this because the tree shapes do not overlap, we have checked that (you can see in the dataset description that the data was collected from satellite images, see: https://data.sfgov.org/Energy-and-Environment/SF-Urban-Tree-Canopy/55pv-5zcc). The sum of these lengths represents the total length over which the street is shaded.

We don't convert CRS because of three reasons:
- CRSes of both geometries are the same
- we calculate percentage so the unit is not important
- the distances around a single city are small so the earth rounding does not play any significant role here

WARNING: this might take a while...

In [9]:
# first bullet point
streets_df.crs == trees_df.crs

True

In [10]:
# the most important cell
# calculates the shaded length of each street going tree by tree
def calculate_shading(row):
    return trees_shapes.intersection(row['geometry']).length.sum()

streets_df['shaded_length'] = streets_df.apply(calculate_shading, axis=1)

KeyboardInterrupt: 

In [ ]:
# calculating the percentage
streets_df['shaded_percent'] = streets_df['shaded_length'] / streets_df['length']

In [ ]:
# saving data
streets_df.to_csv("../data/streets_data_shaded.csv")